In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
aws_data = pd.read_csv("/raid/workspace/honghee/data_weather/train_x/aws/aws_final_2019.csv")
# aws_data
training_x = aws_data.iloc[:,4:].values  # 18(# of features) * 5500000(# of data)
training_y = aws_data.iloc[:,3].values

print(training_x.shape)
print(training_y[0])


(5501280, 18)
['0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0']


In [3]:
# training_data
# multivariate data preparation
from numpy import array
import ast
 
# split a multivariate sequence into samples
def split_sequences(sequences_x, sequence_y,  n_steps):
    X, y = list(), list()
    for i in range(len(sequence_y)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequence_y):
            break
        # gather input and output parts of the pattern
        seq_x = sequences_x[i:end_ix, :]
        seq_y = sequence_y[end_ix-1]
        seq_y = ast.literal_eval(seq_y)
#         print(seq_y)
        seq_y = list(map(int, seq_y))
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)


In [4]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
#         self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 13)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [5]:
n_features = 18 # this is number of parallel inputs
n_timesteps = 12 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(training_x, training_y, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net = MV_LSTM(n_features,n_timesteps)
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=1e-1)

train_episodes = 100
batch_size = 16

(5501269, 12, 18) (5501269, 13)


In [ ]:
mv_net.train()
for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]    
        
        x_batch = torch.tensor(inpt,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)
    
        mv_net.init_hidden(x_batch.size(0))
    #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
    #    lstm_out.contiguous().view(x_batch.size(0),-1)
        output = mv_net(x_batch) 
#         output = torch.reshape(output, (16, 13))
#         loss = criterion(output.view(-1), y_batch)  
        loss = criterion(output, y_batch)  
        
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 
    print('step : ' , t , 'loss : ' , loss.item())

step :  0 loss :  0.014627601020038128
step :  1 loss :  0.014628618955612183
step :  2 loss :  0.014629067853093147
